In [1]:
from datasets import load_from_disk
from pprint import pprint
import json

/home/laptop/EXERCISES/MISCELLANEOUS/miscellaneous/LLM/ARCEE_AGENT/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load the Dataset

In [2]:
ds = load_from_disk("./dataset")
ds

Dataset({
    features: ['query', 'id', 'answers', 'tools'],
    num_rows: 15
})

In [3]:
example = ds[0]
example

{'query': 'Where can I find live giveaways for beta access and games?',
 'id': 0,
 'answers': '[{"name": "live_giveaways_by_type", "arguments": {"type": "beta"}}, {"name": "live_giveaways_by_type", "arguments": {"type": "game"}}]',
 'tools': '[{"name": "live_giveaways_by_type", "description": "Retrieve live giveaways from the GamerPower API based on the specified type.", "parameters": {"type": {"description": "The type of giveaways to retrieve (e.g., game, loot, beta).", "type": "str", "default": "game"}}}]'}

## Data Example

In [4]:
print(f"Query: {example['query']}")
print()
print("Tools:")
pprint(json.loads(example["tools"])[0])
print()
print("Answer:")
pprint(json.loads(example["answers"])[0])

Query: Where can I find live giveaways for beta access and games?

Tools:
{'description': 'Retrieve live giveaways from the GamerPower API based on the '
                'specified type.',
 'name': 'live_giveaways_by_type',
 'parameters': {'type': {'default': 'game',
                         'description': 'The type of giveaways to retrieve '
                                        '(e.g., game, loot, beta).',
                         'type': 'str'}}}

Answer:
{'arguments': {'type': 'beta'}, 'name': 'live_giveaways_by_type'}


**Note: you won't have the "answer" column in our private test set!**

## Example for expected output

In [6]:
example_answer = [{"arguments": {"type": "test_arg"}, "name": "test_tool"}]
example_str = json.dumps(example_answer)
example_str

'[{"arguments": {"type": "test_arg"}, "name": "test_tool"}]'

In [7]:
ds = ds.add_column("my_answers", [example_str for _ in ds["query"]])
ds

Dataset({
    features: ['query', 'id', 'answers', 'tools', 'my_answers'],
    num_rows: 15
})

In [30]:
# This would save the modified dataset to disk
# ds.save_to_disk("./my_dataset")

# Complete Arcee Agent Function Calling Workflow

This section demonstrates the complete workflow for using Arcee Agent with function calling.

In [5]:
# Import our main functions
import sys
sys.path.append('.')
from main import create_function_calling_prompt, parse_tool_calls

# Test with the first example
example = ds[0]
query = example['query']
tools = json.loads(example['tools'])

print("Query:", query)
print("\nTools:")
pprint(tools)

# Create function calling prompt
prompt = create_function_calling_prompt(query, tools)
print("\n" + "="*50)
print("FUNCTION CALLING PROMPT:")
print("="*50)
print(prompt)

Query: Where can I find live giveaways for beta access and games?

Tools:
[{'description': 'Retrieve live giveaways from the GamerPower API based on the '
                 'specified type.',
  'name': 'live_giveaways_by_type',
  'parameters': {'type': {'default': 'game',
                          'description': 'The type of giveaways to retrieve '
                                         '(e.g., game, loot, beta).',
                          'type': 'str'}}}]

FUNCTION CALLING PROMPT:
You are an AI assistant with access to the following tools:

[
  {
    "name": "live_giveaways_by_type",
    "description": "Retrieve live giveaways from the GamerPower API based on the specified type.",
    "parameters": {
      "type": {
        "description": "The type of giveaways to retrieve (e.g., game, loot, beta).",
        "type": "str",
        "default": "game"
      }
    }
  }
]

Based on the user's query, determine which tool(s) to call and with what arguments.
Your response should be a JSON

In [6]:
# Test the tool call parsing with expected response format
expected_response = '''[
  {
    "name": "live_giveaways_by_type",
    "arguments": {
      "type": "beta"
    }
  },
  {
    "name": "live_giveaways_by_type", 
    "arguments": {
      "type": "game"
    }
  }
]'''

print("Expected Model Response:")
print(expected_response)

# Parse the tool calls
parsed_calls = parse_tool_calls(expected_response)
print("\nParsed Tool Calls:")
pprint(parsed_calls)

# Compare with ground truth
ground_truth = json.loads(example['answers'])
print("\nGround Truth:")
pprint(ground_truth)

print(f"\nMatches ground truth: {parsed_calls == ground_truth}")

Expected Model Response:
[
  {
    "name": "live_giveaways_by_type",
    "arguments": {
      "type": "beta"
    }
  },
  {
    "name": "live_giveaways_by_type", 
    "arguments": {
      "type": "game"
    }
  }
]

Parsed Tool Calls:
[{'arguments': {'type': 'beta'}, 'name': 'live_giveaways_by_type'},
 {'arguments': {'type': 'game'}, 'name': 'live_giveaways_by_type'}]

Ground Truth:
[{'arguments': {'type': 'beta'}, 'name': 'live_giveaways_by_type'},
 {'arguments': {'type': 'game'}, 'name': 'live_giveaways_by_type'}]

Matches ground truth: True


## Running the Complete Workflow

To run the complete Arcee Agent function calling workflow:

### 1. Using Ollama (Recommended for beginners)

```bash
# First, install Ollama from https://ollama.ai/
# Then run the demo
python demo_ollama.py
```

### 2. Using the main script with Ollama

```bash
# After Ollama is set up
python main.py --model arcee-ai/arcee-agent --base_url http://127.0.0.1:11434/v1 --max_samples 5 --evaluate
```

### 3. Using VLLM (for production)

```bash
# Install VLLM
pip install vllm

# Start VLLM server
vllm serve arcee-ai/Arcee-Agent --port 8000

# In another terminal
python main.py --model arcee-ai/Arcee-Agent --base_url http://127.0.0.1:8000/v1 --max_samples 5 --evaluate
```

### 4. Using local quantized model

```bash
# Download the model first
python main.py --download_model

# Use local model
python main.py --model arcee-agent-local --use_local_model --max_samples 5
```